In [21]:
## personal imports
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime

from fund.fund import Fund

## risk and return class imports
from returns.products import Products as returns_products_class

## standard imports
from statistics import variance
import math
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [23]:
market = Market()

In [24]:
delta_fund = Fund()
delta_fund.initialize()

In [25]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()

In [26]:
analysis = []
for strategy in tqdm(delta_fund.strategies):
    try:
        parameters = strategy.pull_iterations().to_dict("records")
        for parameter in tqdm(parameters):
            try:
                trade = strategy.db.query("trades",{"iteration":parameter["iteration"]})
                if trade.index.size > 0:
                    positions = int(strategy.positions) if strategy.pricer_class.asset_class.value == "stocks" else 1
                    pricer_timeframe =  strategy.pricer_class.time_horizon_class.naming_convention
                    trade = trade[trade["position"] <= positions]
                    ledger = trade.sort_values("date").copy()
                    ledger = ledger[(ledger["date"]>=delta_fund.start) & (ledger["date"]<=delta_fund.end)]
                    anas = strategy.analysis
                    if ledger.index.size > 0:
                        cumulative = anas.trade_analysis(ledger,positions)
                        for key in parameter.keys():
                            cumulative[key] = parameter[key]
                        cumulative["strategy"] = strategy.name
                        analysis.append(cumulative[list(parameter.keys()) + ["strategy","pv"]].iloc[-1])
            except Exception as e:
                print(str(e))
                continue
        strategy.db.disconnect()
    except Exception as e:
        print(str(e))
a = pd.DataFrame([x.to_dict() for x in analysis])

 28%|███████████████████████████████████████████████▌                                                                                                                         | 36/128 [00:08<00:20,  4.56it/s]


 58%|█████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                       | 74/128 [00:18<00:14,  3.68it/s]


 88%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                     | 112/128 [00:28<00:04,  3.53it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.72s/it]


In [37]:
# a.sort_values("pv",ascending=False).head(20)

,constituent,value,ceiling,classification,rank,floor,buy_day,iteration,strategy,pv
127,500,False,10,False,False,0,1,127,strategy_dsrp_qsrc_qser_b_r,6.890645
126,500,False,1,False,False,0,1,126,strategy_dsrp_qsrc_qser_b_r,6.890645
125,500,False,10,False,False,-10,1,125,strategy_dsrp_qsrc_qser_b_r,6.890645
124,500,False,1,False,False,-10,1,124,strategy_dsrp_qsrc_qser_b_r,6.890645
108,500,False,1,False,False,-10,1,108,strategy_dsrp_qsrc_qser_b_r,6.890645
109,500,False,10,False,False,-10,1,109,strategy_dsrp_qsrc_qser_b_r,6.890645
110,500,False,1,False,False,0,1,110,strategy_dsrp_qsrc_qser_b_r,6.890645
111,500,False,10,False,False,0,1,111,strategy_dsrp_qsrc_qser_b_r,6.890645
43,100,False,10,False,True,0,1,43,strategy_dsrp_qsrc_qser_b_r,5.450867
41,100,False,10,False,True,-10,1,41,strategy_dsrp_qsrc_qser_b_r,5.450867


In [28]:
top_by_strategy = a[a["rank"]==False].sort_values("pv",ascending=False).groupby("strategy").nth(0).reset_index().sort_values("pv",ascending=False)
# top_by_strategy

In [29]:
for row in top_by_strategy.iterrows():
    algo_name = row[1]["strategy"]
    parameter = row[1]
    db = ADatabase(algo_name)
    db.connect()
    db.drop("optimal")
    db.store("optimal",pd.DataFrame([row[1].to_dict()]))
    db.disconnect()
    db.cloud_connect()
    db.drop("optimal")
    db.store("optimal",pd.DataFrame([row[1].to_dict()]))
    db.disconnect()

In [30]:
cumulatives = []
ledgers = []
for row in top_by_strategy.iterrows():
    name = row[1]["strategy"]
    iteration = row[1]["iteration"]
    db = ADatabase(name)
    db.connect()
    analysis = db.query("trades",{"iteration":int(iteration)})
    db.disconnect()
    ledger = analysis[["date","ticker","actual_returns","position"]]
    ledger = ledger[(ledger["date"]>=delta_fund.start) & (ledger["date"]<=delta_fund.end)]
    ledger = ledger.merge(sp500[["ticker","GICS Sector","Security"]],on="ticker",how="left")
    ledger["strategy"] = name
    ledgers.append(ledger)
    portfolio = ledger.pivot_table(index="date",columns="position",values="actual_returns").fillna(1).reset_index()
    positions = ledger["position"].max()
    cumulative = anas.iteration_analysis(portfolio,positions)
    cumulative["strategy"] = name
    for i in range(positions):
        cumulative.rename(columns={i:str(i)},inplace=True)
    db.cloud_connect()
    db.drop("performance")
    db.store("performance",cumulative)
    db.disconnect()
    cumulatives.append(cumulative)

In [31]:
portfolio = pd.concat(cumulatives).pivot_table(index="date",columns="strategy",values="pv").fillna(method="ffill").fillna(1).reset_index()
portfolio["pv"] = [sum([row[1][strategy]*(1/top_by_strategy.index.size) for strategy in portfolio.columns if strategy not in ["date"]]) for row in portfolio.iterrows()]

In [32]:
portfolio[["date","pv"]].tail(1)

In [33]:
# plt.plot(portfolio["date"],portfolio["pv"])

In [35]:
# all_ledgers = pd.concat(ledgers)

In [36]:
# all_ledgers.groupby(["strategy","GICS Sector"]).mean().reset_index().sort_values("actual_returns",ascending=False)